In [1]:
import gc
import numpy as np
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
import utils, segmentation
from tqdm.auto import tqdm

In [2]:
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')
print(device)

cuda:1


In [3]:
class PartialDs(torch.utils.data.Dataset):
    def __init__(self, ds, idxs):
        self.ds = ds
        self.idxs = idxs
    
    def __len__(self):
        return len(self.idxs)
    
    def __getitem__(self, i): 
        return self.ds[self.idxs[i]]

def create_model():
    return smp.Unet(
        encoder_name="resnet18",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=2,                      # model output channels (number of classes in your dataset)
    )

def create_optimizer(model):
    return torch.optim.SGD(model.parameters(), lr=1e-1)

In [4]:
ds_full = segmentation.SegmentationDataset('./stenoses_data/train/', 2)
ds_test = segmentation.SegmentationDataset('./stenoses_data/test/', 2)

batch_size = 8
dl_full = torch.utils.data.DataLoader(ds_full, batch_size=batch_size, shuffle=False, drop_last=False)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, shuffle=False, drop_last=False)

In [5]:
w = 0.
for imgs, masks in dl_full:
    w += masks.sum() / (masks.numel() * len(dl_full))
loss_fn = nn.CrossEntropyLoss(weight=torch.Tensor([1 / (1-w), 1/ w])).to(device)

In [6]:
model = create_model().to(device)
optimizer = create_optimizer(model)

In [7]:
N = len(ds_full)
N_step = int(N*0.1)
N, N_step

(598, 59)

In [8]:
idxs = list(range(N))
np.random.seed(0)
np.random.shuffle(idxs)

In [9]:
for n_iter in range(10):
    print(f'>>>>>>>>>>>> Iter {n_iter}')
    del model
    del optimizer
    gc.collect()
    labeled_idxs = idxs[:N_step*(n_iter+1)]
    ds_train = PartialDs(ds_full, labeled_idxs)
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, drop_last=True)

    model = create_model().to(device)
    optimizer = create_optimizer(model)

    num_epochs = 30
    for epoch in tqdm(range(num_epochs), desc='Epoch'):
        trn_loss = utils.train(dl_train, loss_fn, model, optimizer)
        val_loss, val_IoUs = segmentation.evaluate(dl_test, loss_fn, model)
        print(epoch, f'{trn_loss:.4f}', f'{val_loss:.4f}', val_IoUs)
    print()

>>>>>>>>>>>> Iter 0


Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

0 0.4571 1.8557 tensor([0.0024])
1 0.2324 1.4524 tensor([0.0052])
2 0.1826 0.3398 tensor([0.0085])
3 0.1570 0.2998 tensor([0.0103])
4 0.1024 0.2925 tensor([0.0272])
5 0.0667 0.1586 tensor([0.0863])
6 0.0413 0.3316 tensor([0.1308])
7 0.0288 0.2890 tensor([0.1666])
8 0.0280 0.1672 tensor([0.0607])
9 0.0223 0.1043 tensor([0.0771])
10 0.0195 0.4539 tensor([0.1302])
11 0.0398 0.0819 tensor([0.0641])
12 0.0423 0.6755 tensor([0.0098])
13 0.0612 0.1405 tensor([0.0487])
14 0.0237 0.5028 tensor([0.0989])
15 0.0427 0.0821 tensor([0.0293])
16 0.0298 0.1822 tensor([0.0858])
17 0.0220 0.2859 tensor([0.1123])
18 0.0184 0.3837 tensor([0.1588])
19 0.0157 0.6191 tensor([0.1589])
20 0.0131 0.7149 tensor([0.1495])
21 0.0114 0.9659 tensor([0.1259])
22 0.0104 0.8656 tensor([0.1391])
23 0.0096 0.7203 tensor([0.1507])
24 0.0084 1.2477 tensor([0.1197])
25 0.0075 1.2970 tensor([0.1323])
26 0.1855 7.5379 tensor([0.0026])
27 0.0981 0.3428 tensor([0.0106])
28 0.0596 0.1245 tensor([0.0450])
29 0.0420 0.1521 tensor(

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

0 0.6060 45.0837 tensor([0.0018])
1 0.3484 0.1699 tensor([0.0140])
2 0.0936 0.1066 tensor([0.0341])
3 0.0505 0.0817 tensor([0.0396])
4 0.0551 0.1432 tensor([0.0967])
5 0.0234 0.3747 tensor([0.2221])
6 0.0542 0.1971 tensor([0.0138])
7 0.0374 0.0871 tensor([0.0970])
8 0.0205 0.0872 tensor([0.1718])
9 0.0184 0.1300 tensor([0.2656])
10 0.0157 0.1563 tensor([0.3081])
11 0.0109 0.1112 tensor([0.2042])
12 0.0087 0.1234 tensor([0.2875])
13 0.0079 0.1859 tensor([0.3405])
14 0.0071 0.0997 tensor([0.2087])
15 0.0070 0.0950 tensor([0.2298])
16 0.0086 0.1654 tensor([0.3693])
17 0.0067 0.2082 tensor([0.3692])
18 0.0055 0.2116 tensor([0.3937])
19 0.0054 0.2618 tensor([0.4202])
20 0.0050 0.2577 tensor([0.4057])
21 0.0050 0.1985 tensor([0.3630])
22 0.0045 0.1554 tensor([0.3625])
23 0.0042 0.1872 tensor([0.3794])
24 0.0122 0.1670 tensor([0.3295])
25 0.0059 0.2948 tensor([0.3939])
26 0.0045 0.2873 tensor([0.4076])
27 0.0046 0.2941 tensor([0.4267])
28 0.0040 0.3315 tensor([0.4364])
29 0.0039 0.2775 tensor

...

0 0.1370 0.2510 tensor([0.2826])
1 0.0745 0.0587 tensor([0.1066])
2 0.0207 0.1725 tensor([0.4162])
3 0.0373 0.0544 tensor([0.0624])
4 0.0176 0.0671 tensor([0.3339])
5 0.0169 0.0802 tensor([0.4093])
6 0.0090 0.0298 tensor([0.1298])
7 0.0072 0.0217 tensor([0.3237])
8 0.0055 0.0633 tensor([0.4791])
9 0.0051 0.0320 tensor([0.3792])
10 0.0051 0.0554 tensor([0.4731])
11 0.0044 0.0887 tensor([0.5258])
12 0.0054 0.0862 tensor([0.5340])
13 0.0037 0.0367 tensor([0.4755])
14 0.0045 0.1145 tensor([0.5530])
15 0.0037 0.1043 tensor([0.5810])
16 0.0036 0.0441 tensor([0.4274])
17 0.0037 0.0500 tensor([0.3956])
18 0.0033 0.1604 tensor([0.5882])
19 0.0031 0.0692 tensor([0.5577])
20 0.0027 0.1070 tensor([0.5638])
21 0.0030 0.2465 tensor([0.6534])
22 0.0030 0.1622 tensor([0.6280])
23 0.0032 0.1008 tensor([0.5759])
24 0.0029 0.0734 tensor([0.5164])
25 0.0027 0.1936 tensor([0.6265])
26 0.0024 0.2202 tensor([0.6212])
27 0.0024 0.1765 tensor([0.6285])
28 0.0027 0.2307 tensor([0.6362])
29 0.0024 0.2489 tensor(

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

0 0.1372 0.1356 tensor([0.3113])
1 0.0144 0.0510 tensor([0.3007])
2 0.0096 0.1763 tensor([0.5076])
3 0.0069 0.0716 tensor([0.3916])
4 0.0058 0.0411 tensor([0.4242])
5 0.0051 0.1128 tensor([0.5068])
6 0.0043 0.1320 tensor([0.5372])
7 0.0035 0.1800 tensor([0.5835])
8 0.7794 0.6610 tensor([0.0167])
9 0.2081 0.5551 tensor([0.1203])
10 0.0446 0.1358 tensor([0.0847])
11 0.0191 0.3081 tensor([0.2146])
12 0.0132 0.2954 tensor([0.1351])
13 0.0101 0.5533 tensor([0.3189])
14 0.0082 0.1937 tensor([0.1872])
15 0.0082 0.2067 tensor([0.2819])
16 0.0060 0.2688 tensor([0.3819])
17 0.0075 0.3131 tensor([0.3250])
18 0.0059 0.2340 tensor([0.3652])
19 0.0055 0.2872 tensor([0.3469])
20 0.0050 0.2836 tensor([0.3696])
21 0.0045 0.2492 tensor([0.3893])
22 0.0045 0.3696 tensor([0.4056])
23 0.0041 0.2401 tensor([0.3137])
24 0.0047 0.2754 tensor([0.3991])
25 0.0044 0.3383 tensor([0.4447])
26 0.0039 0.3196 tensor([0.4235])
27 0.0039 0.3303 tensor([0.3975])
28 0.0036 0.3452 tensor([0.4272])
29 0.0036 0.3066 tensor(

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

0 0.1634 0.1159 tensor([0.0213])
1 0.0286 0.1450 tensor([0.0299])
2 0.0155 0.1590 tensor([0.2512])
3 0.0090 0.1300 tensor([0.0718])
4 0.0069 0.2851 tensor([0.3832])
5 0.0058 0.3977 tensor([0.4145])
6 0.0047 0.3593 tensor([0.3964])
7 0.0043 0.4222 tensor([0.4334])
8 0.0042 0.5226 tensor([0.4178])
9 0.0036 0.3227 tensor([0.3927])
10 0.0037 0.5332 tensor([0.4202])
11 0.0031 0.4034 tensor([0.4740])
12 0.0030 0.4113 tensor([0.3995])
13 0.0029 0.4095 tensor([0.4219])
14 0.0032 0.3531 tensor([0.4627])
15 0.0030 0.3750 tensor([0.4657])
16 0.0027 0.4949 tensor([0.4585])
17 0.0026 0.3753 tensor([0.4707])
18 0.0023 0.2968 tensor([0.4146])
19 0.0023 0.5324 tensor([0.4551])
20 0.0022 0.4745 tensor([0.4748])
21 0.0022 0.4011 tensor([0.4663])
22 0.0022 0.5349 tensor([0.4823])
23 0.0021 0.5446 tensor([0.4719])
24 0.0020 0.5418 tensor([0.4960])
25 0.0020 0.4030 tensor([0.4569])
26 0.0026 0.3308 tensor([0.2549])
27 0.0030 0.4416 tensor([0.4807])
28 0.0035 0.2308 tensor([0.4232])
29 0.0025 0.3406 tensor(